In [1]:
import numpy as np
import pandas as pd
import os
from function import Slope, minSSE_recovery

s_c = '結算價'
top_path = './../../../'
Data_path = top_path + 'InterpData/'
expiry = 'NearbyMonth'

IV_type = 'callIV'
K_Range = [300, 500]
K_Range_file = '{}_{}.csv'.format(K_Range[0], K_Range[1])
K_Range_dir = '{}_{}'.format(K_Range[0], K_Range[1])
IV_path = '{}/{}/{}/{}'.format(Data_path, expiry, IV_type, K_Range_file)

Dir_tree = [top_path, 'ForecastData', expiry, IV_type, K_Range_dir]
current_path = Dir_tree[0]
for i in range(1, len(Dir_tree), 1):
    if Dir_tree[i] not in os.listdir(current_path):
        os.mkdir(current_path + Dir_tree[i])
    current_path = current_path + Dir_tree[i] + '/'

IV_data = pd.read_csv(IV_path, encoding='Big5', index_col=False)
IV_matrix = np.array(IV_data)

#----------------------------------------------------------------------------


OC = np.array(IV_data['期貨開盤價'] - IV_data['期貨收盤價'])
HL = np.array(IV_data['期貨最高價'] - IV_data['期貨最低價'])
expirty_days = np.array(IV_data['到期天數'])
#OC_HL_K_E = np.vstack((OC, HL, Kmin_reduce_F, Kmax_reduce_F, expirty_days)).T
OCHL = np.vstack((OC, HL)).T


IV_matrix = np.array(IV_data)
K_num = len(np.where(IV_matrix[0, 0] == IV_matrix)[0])
K = np.array(IV_data['履約價'])
K = np.reshape(K, (-1, K_num))
IV= np.array(IV_data['隱含波動率({})'.format(s_c)])
IV = np.reshape(IV, (-1, K_num))
K_IVslope = Slope(X=K, Y=IV, axis=1)
E = IV_matrix[range(0, len(IV_matrix), K_num), IV_data.columns.get_loc('到期天數')]

In [2]:
import tsia
def _markov_transition_field(X_binned, X_mtm, n_timestamps):
    X_mtf = np.zeros((n_timestamps, n_timestamps))
    
    # We loop through each timestamp twice to build a N x N matrix:
    for i in range(n_timestamps):
        for j in range(n_timestamps):
            # We align each probability along the temporal order: MTF(i,j) 
            # denotes the transition probability of the bin 'i' to the bin 
            # 'j':
            X_mtf[i, j] = X_mtm[X_binned[i], X_binned[j]]
            
    return X_mtf


one_day_expiry_idx = np.arange(len(E))[np.equal(E, 1)]
if one_day_expiry_idx[-1] == len(E)-1:
    one_day_expiry_idx = one_day_expiry_idx[:-1]
most_days_expiry_idx = one_day_expiry_idx + 1
most_days_expiry_idx = np.hstack((0, most_days_expiry_idx))

most_days_expiry_idx0 = most_days_expiry_idx[:-1]
most_days_expiry_idx1 = most_days_expiry_idx[1:]
trade_days_in_month = most_days_expiry_idx1 - most_days_expiry_idx0  
most_days_expiry = E[most_days_expiry_idx]

contract_appear_days1 = [list(range(1, trade_day_in_month+1, 1)) \
                for trade_day_in_month in trade_days_in_month]
contract_appear_days1 = [contract_appear_day1 for subcontract_appear_day1 in contract_appear_days1\
                         for contract_appear_day1 in subcontract_appear_day1]
contract_appear_days1 = np.array(contract_appear_days1)
contract_appear_days2 = np.arange(1, len(E) - most_days_expiry_idx[-1] +1, 1)
contract_appear_days = np.hstack((contract_appear_days1, contract_appear_days2))
#contract_appear_days 為該契約(特定交易日期、到期日期，履約價不限)上市的交易日數，例如第一天上市到期天數35天，
#則值為1，第二天則到期日為34天值為2，一直到k(因為是交易日數，所以數字不一定)。然後又到下個月的契約，值又從1開
#始


from function import TimeSeriesData
seq_length1 = 14
seq_length2 = seq_length1 + 1
min_contract_day = 2
magnification_slope = 10000
Q = 4

#變數有考慮到期日
#K_IVslope_E = np.hstack((K_IVslope*magnification_slope, np.reshape(E, (-1, 1))))

#Inputs_slope, Ouputs = TimeSeriesData(K_IVslope_E, seq_length=seq_length1, \
                                      #drop_out_columns=[len(K_IVslope_E[0])-1])

#變數沒考慮到期日
K_IVslope_E = K_IVslope * magnification_slope
Inputs_slope, Ouputs = TimeSeriesData(K_IVslope_E, seq_length=seq_length1, drop_out_columns=[])



Inputs_OC  = OC[range(0, len(IV_matrix), K_num)]
Inputs_OC  = np.array([Inputs_OC[i:i+seq_length2] \
                              for i in range(len(Inputs_OC)-seq_length2+1)])
Inputs_OC  = Inputs_OC.astype(float)
Inputs_HL  = HL[range(0, len(IV_matrix), K_num)]
Inputs_HL  = np.array([Inputs_HL[i:i+seq_length2] \
                              for i in range(len(Inputs_HL)-seq_length2+1)])
Inputs_HL  = Inputs_HL.astype(float)
if seq_length1  > seq_length2-1:
    Inputs_OC = Inputs_OC[seq_length1-seq_length2+1:]
    Inputs_HL = Inputs_HL[seq_length1-seq_length2+1:]
    contract_appear_days = contract_appear_days[seq_length1-seq_length2+1:]
    IV_matrix_forecast = IV_matrix[seq_length1*K_num:]
if seq_length1 <= seq_length2-1:
    Inputs_slope = Inputs_slope[seq_length2-1-seq_length1:]
    Ouputs = Ouputs[seq_length2-1-seq_length1:]
    contract_appear_days = contract_appear_days[seq_length2-1:]
    IV_matrix_forecast = IV_matrix[(seq_length2-1)*K_num:]


match_cond = np.greater_equal(contract_appear_days, min_contract_day)
Inputs_OC = Inputs_OC[match_cond]
Inputs_HL = Inputs_HL[match_cond]
Inputs_slope = Inputs_slope[match_cond]
contract_appear_days_filter = contract_appear_days[match_cond]
contract_appear_days_filter_Knum = contract_appear_days_filter.repeat(K_num)
match_cond_Knum = match_cond.repeat(K_num)
IV_matrix_forecast = IV_matrix_forecast[match_cond_Knum]
IV_matrix_forecast= np.hstack((IV_matrix_forecast, np.reshape(contract_appear_days_filter_Knum, (-1, 1))))
Ouputs = Ouputs[match_cond]


GGM_OC = np.zeros((len(Inputs_OC), seq_length2, seq_length2, 3))
for i in range(len(Inputs_OC)):
    X = Inputs_OC[i]
    X_hat = ((X-X.max()) + (X-X.min())) / (X.max() - X.min())
    X_hat = np.reshape(X_hat,(1,-1))
    y = np.sqrt(np.ones((1,X_hat.size)) - X_hat**2)
    Gcos = X_hat.T @ X_hat - (y.T @ y)

    y = np.sqrt(np.ones((1,X_hat.size)) - X_hat**2)
    Gsin = y.T @ X_hat + (y.T @ X_hat).T

    X_ = pd.DataFrame(X)
    X_binned, bin_edges = tsia.markov.discretize(X_, Q)

    X_mtm = tsia.markov.markov_transition_matrix(X_binned)

    X_mtm = tsia.markov.markov_transition_probabilities(X_mtm)

    X_mtf = _markov_transition_field(X_binned, X_mtm, seq_length2)
    
    GGM_OC[i, :,:,0] = Gcos 
    GGM_OC[i, :,:,1] = Gsin 
    GGM_OC[i, :,:,2] = X_mtf 

GGM_HL = np.zeros((len(Inputs_HL), seq_length2, seq_length2, 3))
for i in range(len(Inputs_HL)):
    X = Inputs_HL[i]
    X_hat = ((X-X.max()) + (X-X.min())) / (X.max() - X.min())
    X_hat = np.reshape(X_hat,(1,-1))
    y = np.sqrt(np.ones((1,X_hat.size)) - X_hat**2)
    Gcos = X_hat.T @ X_hat - (y.T @ y)

    y = np.sqrt(np.ones((1,X_hat.size)) - X_hat**2)
    Gsin = y.T @ X_hat + (y.T @ X_hat).T

    X_ = pd.DataFrame(X)
    X_binned, bin_edges = tsia.markov.discretize(X_, Q)

    X_mtm = tsia.markov.markov_transition_matrix(X_binned)

    X_mtm = tsia.markov.markov_transition_probabilities(X_mtm)

    X_mtf = _markov_transition_field(X_binned, X_mtm, seq_length2)
    
    GGM_HL[i, :,:,0] = Gcos 
    GGM_HL[i, :,:,1] = Gsin 
    GGM_HL[i, :,:,2] = X_mtf 

train_size = int(len(Inputs_slope)*0.8)
X_train = Inputs_slope[:train_size]
OC_train = GGM_OC[:train_size ]
HL_train = GGM_HL[:train_size ]
y_train = Ouputs[:train_size]



X_test = Inputs_slope[train_size:]
y_test = Ouputs[train_size:]
IV_matrix_test = IV_matrix_forecast[K_num*train_size:]
OC_test = GGM_OC[train_size:]
HL_test = GGM_HL[train_size:]

In [3]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, concatenate, Conv2D, Flatten
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'

input = Input(shape=OC_train.shape[1:])
hidden1 = Conv2D(filters=32,
                     kernel_size=(3,3),
                     strides=(1,1),
                     padding='same',
                     activation='relu',
                     )(input)
hidden2 = Conv2D(filters=64,
                     kernel_size=(5,5),
                     strides=(1,1),
                     padding='same',
                     activation='relu',
                     )(hidden1)
output = Flatten()(hidden2)
output = Dense(y_train.shape[1])(output)
model = Model(inputs=input, outputs=output)

model.compile(loss='mse', optimizer = 'adam', metrics=['mse'])  


early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=10, \
                               verbose=2, mode='min', restore_best_weights=True)
hist_model = model.fit(
        OC_train, y_train, epochs=1000, batch_size=32, \
        validation_split=0.2, callbacks=[early_stopping]
)
plot_model(model, to_file='cnn.png', show_shapes=True, show_layer_names=True)


Epoch 1/1000


: 

In [3]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, concatenate, Conv2D, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'
#此函示可以幫我們設定一個模型的結構
def set_CNN_LSTM(lstm_structure, cnn1_structure, cnn2_structure, Megred_struture):
    lstm_neurons_num = lstm_structure['lstm_neurons_num']
    lstm_activations = lstm_structure['lstm_activations']
    lstm_recurrent_activations = lstm_structure['lstm_recurrent_activations']
    dense1_neurons_num = lstm_structure['dense_neurons_num']
    dense1_activations = lstm_structure['dense_activations']
    kernel_regularizer1= lstm_structure['kernel_regularizer']

    cnn1_kernel_size = cnn1_structure['cnn_kernel_size']
    cnn1_filters = cnn1_structure['cnn_filters']
    cnn1_strides = cnn1_structure['cnn_stride']
    cnn1_padding = cnn1_structure['cnn_padding']
    cnn1_activations = cnn1_structure['cnn_activations']
    dense2_neurons_num = cnn1_structure['dense_neurons_num']
    dense2_activations = cnn1_structure['dense_activations']
    kernel_regularizer2= cnn1_structure['kernel_regularizer']

    cnn2_kernel_size = cnn2_structure['cnn_kernel_size']
    cnn2_filters = cnn2_structure['cnn_filters']
    cnn2_strides = cnn2_structure['cnn_stride']
    cnn2_padding = cnn2_structure['cnn_padding']
    cnn2_activations = cnn2_structure['cnn_activations']
    dense3_neurons_num = cnn2_structure['dense_neurons_num']
    dense3_activations = cnn2_structure['dense_activations']
    kernel_regularizer3= cnn2_structure['kernel_regularizer']

    
    Merged_neurons_num = Megred_struture['neurons_num']
    Megred_activations = Megred_struture['activations']
    Merged_kernel_regularizer = Megred_struture['kernel_regularizer']

    input_lstm = Input(shape=lstm_structure['input_shape'])
    hidden_1 = input_lstm

    for i in range(0, len(lstm_neurons_num), 1):
        if i == len(lstm_neurons_num) - 1:
            return_sequences=False
        else:
            return_sequences=True 
        hidden_1 = LSTM(lstm_neurons_num[i], return_sequences=return_sequences,\
        activation=lstm_activations[i], kernel_regularizer=kernel_regularizer1[i],
        recurrent_activation= lstm_recurrent_activations[i])(hidden_1)
    output_1 = hidden_1
    for i in range(0, len(dense1_neurons_num), 1):
        output_1 = Dense(dense1_neurons_num[i], \
            kernel_regularizer=kernel_regularizer1[len(lstm_neurons_num)-1+i], \
            activation=dense1_activations[i])(output_1)
        
    input_cnn1 = Input(shape=cnn1_structure['input_shape'])
    hidden_2 = input_cnn1

    for i in range(0, len(cnn1_kernel_size), 1):
        hidden_2 =  Conv2D(filters=cnn1_filters[i],
                           kernel_size=cnn1_kernel_size[i],
                           strides=cnn1_strides[i],
                           padding=cnn1_padding[i],
                           activation=cnn1_activations[i],
                           kernel_regularizer=kernel_regularizer2[i]
                        )(hidden_2)
    output_2 = Flatten()(hidden_2)
    for i in range(0, len(dense2_neurons_num), 2):
        output_2 = Dense(dense2_neurons_num[i], \
            kernel_regularizer=kernel_regularizer2[len(cnn1_kernel_size)-1+i], \
            activation=dense2_activations[i])(output_2)
    input_cnn2 = Input(shape=cnn2_structure['input_shape'])
    hidden_3 = input_cnn2
    
    for i in range(0, len(cnn2_kernel_size), 1):
        hidden_3 =  Conv2D(filters=cnn2_filters[i],
                           kernel_size=cnn2_kernel_size[i],
                           strides=cnn2_strides[i],
                           padding=cnn2_padding[i],
                           activation=cnn2_activations[i],
                           kernel_regularizer=kernel_regularizer3[i]
                        )(hidden_3)
    output_3 = Flatten()(hidden_3)
    for i in range(0, len(dense3_neurons_num), 2):
        output_3 = Dense(dense3_neurons_num[i], \
            kernel_regularizer=kernel_regularizer3[len(cnn2_kernel_size)-1+i], \
            activation=dense3_activations[i])(output_3)
    merged = concatenate([output_1, output_2, output_3], axis=-1)
    dense = merged
    for i in range(len(Merged_neurons_num)):
        dense = Dense(Merged_neurons_num[i], Megred_activations[i], Merged_kernel_regularizer[i])(dense)

    output = Dense(y_train.shape[1])(dense)
    model = Model(inputs=[input_lstm, input_cnn1, input_cnn2], outputs=output)
    return model






#決定這次的模型編號
model_idx = 1
#本次預測的變數為：slope(隱波與履約價之間的斜率)或iv(隱波的數值)
forecast_variable = 'slope'
#資料設定都相同的情況下，最多儲存多少種模型結果
max_model = 20

#設定參數
lambda1 = 0.01
lambda2 = 0.01
lambda3 = 0.01
lambda4 = 0.01
batch_size = 32
epochs = 1000

Slope_lstm_neurons_num = [600, 600, 600]
Slope_lstm_activations = ['tanh']*len(Slope_lstm_neurons_num)
Slope_lstm_recurrent_activations = ['sigmoid']*len(Slope_lstm_neurons_num)
Slope_dense_neurons_num = [600]
Slope_dense_activations = ['relu']
Slope_kernel_regularizer=[l2(lambda2)]*(len(Slope_lstm_neurons_num) + len(Slope_dense_neurons_num))

OC_cnn_kernel_size = [(3, 3), (5, 5), (3, 3)]
OC_cnn_filters = [32, 64, 32]
OC_cnn_strides = [(1, 1), (1, 1), (1, 1)]
OC_cnn_padding = ['same']*len(OC_cnn_kernel_size)
OC_cnn_activations = ['relu']*len(OC_cnn_kernel_size)
OC_dense_neurons_num = [600]
OC_dense_activations = ['relu']
OC_kernel_regularizer=[l2(lambda2)]*(len(OC_cnn_kernel_size) + len(OC_dense_neurons_num ))

HL_cnn_kernel_size = [(3, 3), (5, 5), (3, 3)]
HL_cnn_filters = [32, 64, 32]
HL_cnn_strides = [(1, 1), (1, 1), (1, 1)]
HL_cnn_padding = ['same']*len(HL_cnn_kernel_size)
HL_cnn_activations = ['relu']*len(HL_cnn_kernel_size)
HL_dense_neurons_num = [600]
HL_dense_activations = ['relu']
HL_kernel_regularizer=[l2(lambda3)]*(len(HL_cnn_kernel_size) + len(HL_dense_neurons_num ))



Merged_neurons_num = [600, 600]
Megred_activations = ['relu']*len(Merged_neurons_num)
Merged_kernel_regularizer=[l2(lambda3)]*(len(Merged_neurons_num))

lstm_Slope_structure = {
    'input_shape': X_train.shape[1:],
    'lstm_neurons_num': Slope_lstm_neurons_num,
    'lstm_activations': Slope_lstm_activations,
    'lstm_recurrent_activations':Slope_lstm_recurrent_activations,
    'dense_neurons_num':Slope_dense_neurons_num,
    'dense_activations':Slope_dense_activations,
    'kernel_regularizer':Slope_kernel_regularizer
}

OC_cnn_structure = {
    'input_shape': OC_train.shape[1:],
    'cnn_kernel_size': OC_cnn_kernel_size,
    'cnn_filters': OC_cnn_filters,
    'cnn_stride': OC_cnn_strides,
    'cnn_padding': OC_cnn_padding,
    'cnn_activations': OC_cnn_activations,
    'dense_neurons_num': OC_dense_neurons_num,
    'dense_activations': OC_dense_activations,
    'kernel_regularizer': OC_kernel_regularizer
}

HL_cnn_structure = {
    'input_shape': HL_train.shape[1:],
    'cnn_kernel_size': HL_cnn_kernel_size,
    'cnn_filters': HL_cnn_filters,
    'cnn_stride': HL_cnn_strides,
    'cnn_padding': HL_cnn_padding,
    'cnn_activations': HL_cnn_activations,
    'dense_neurons_num': HL_dense_neurons_num,
    'dense_activations': HL_dense_activations,
    'kernel_regularizer': HL_kernel_regularizer
}

Merged_structure = {
    'neurons_num': Merged_neurons_num,
    'activations': Megred_activations,
    'kernel_regularizer': Merged_kernel_regularizer
}

alltrain = 3
allMSE = np.zeros(alltrain) 
allForecastIV = np.zeros((K_num*len(OC_test), 3))
allSSE_everyday = np.zeros((K_num*len(OC_test), 3))
allSSE_everyday_adj = np.zeros((K_num*len(OC_test), 3))
for j in range(alltrain):
    #設定模型結構
    model = set_CNN_LSTM(lstm_Slope_structure, OC_cnn_structure, HL_cnn_structure, Merged_structure)
    model.compile(loss='mse', optimizer = 'adam', metrics=['mse'])  

    # 訓練模型
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=10, \
                               verbose=2, mode='min', restore_best_weights=True)
    hist_model = model.fit(
        [X_train, OC_train, HL_train], y_train, epochs=epochs, batch_size=batch_size, \
        validation_split=0.2, callbacks=[early_stopping]
    )




    #得到輸出
    y_pred = model.predict([X_test, OC_train, HL_train])
    y_pred = y_pred / magnification_slope
    columns_names = ['履約價', '隱含波動率({})'.format(s_c)]
    column_index = [IV_data.columns.get_loc(col) for col in columns_names]
    ForecastIV = np.zeros(((len(y_pred), K_num)))
    SSE_everyday = np.zeros(((len(y_pred), K_num)))
    SSE_everyday_adj = np.zeros(((len(y_pred), K_num)))

    model_name = 'model{}'.format(model_idx)
    model_file = 'model{}.h5'.format(model_idx)
    model_png = 'model{}.png'.format(model_idx)
    Forecast_name = 'model{}.csv'.format(model_idx)
    
    for i in range(len(y_pred)):
        K_day = IV_matrix_test[i*K_num:(i+1)*K_num, column_index[0]]
        iv_day = IV_matrix_test[i*K_num:(i+1)*K_num, column_index[1]]
        y_pred_day = y_pred[i]
        iv_day_std = iv_day / np.std(iv_day, ddof=1)
        y_pred_day_adj = y_pred_day / np.std(iv_day, ddof=1)
        ForecastIV[i], SSE_everyday[i] = minSSE_recovery(y=iv_day, x=K_day, slope_yhat=y_pred_day)
        ForecastIV_day_adj, SSE_everyday_adj[i] = minSSE_recovery(y=iv_day_std, x=K_day, slope_yhat=y_pred_day_adj)


    ForecastIV = np.reshape(ForecastIV,(-1,1))
    sse_everyday = SSE_everyday[:, 0]
    sse_everyday_adj = SSE_everyday_adj[:, 0]
    SSE_everyday = np.reshape(SSE_everyday ,(-1,1))
    SSE_everyday_adj = np.reshape(SSE_everyday_adj ,(-1,1))
    allMSE[j] = np.mean(sse_everyday)
    allForecastIV[:,j:j+1] = ForecastIV
    allSSE_everyday[:, j:j+1] = SSE_everyday
    allSSE_everyday_adj[:, j:j+1] = SSE_everyday_adj

min_MSE_idx = np.argsort(allMSE)[0]
ForecastIV = allForecastIV[:, min_MSE_idx:min_MSE_idx+1]
allSSE_everyday = allSSE_everyday[:, min_MSE_idx:min_MSE_idx+1]
allSSE_everyday_adj = allSSE_everyday_adj[:, min_MSE_idx:min_MSE_idx+1]
min_MSE = allMSE[min_MSE_idx]

Forecast_matrix  = np.hstack((IV_matrix_test, ForecastIV, SSE_everyday, SSE_everyday_adj))
column = np.hstack((IV_data.columns.to_numpy(), \
                        np.array(['上市天數(交易日)','預測隱含波動率({})'.format(s_c), 'loss', '調整後loss'])))
    
   
Forecast_Data = pd.DataFrame(data=Forecast_matrix, columns=column)


loss_columns_names = ['交易日期', '到期天數', '上市天數(交易日)', 'loss']
loss_columns_index = [Forecast_Data.columns.get_loc(col) for col in loss_columns_names]
loss_Data = Forecast_Data.iloc[range(0, len(Forecast_Data), K_num), loss_columns_index]
loss_Data = loss_Data.reset_index().iloc[:,1:]
loss_Data = loss_Data.rename(columns={'loss': model_name})
MSE_data = pd.DataFrame(columns=loss_Data.columns, data=[['MSE', 'MSE', 'MSE', min_MSE]])
loss_Data = pd.concat([loss_Data, MSE_data], axis=0)
loss_Data = loss_Data.reset_index().iloc[:,1:]
print(min_MSE)




model_path = top_path
model_type = 'CNN-LSTM'

model_Dir_tree = ['Forecast&model', expiry, IV_type, 'K_{}'.format(K_Range_dir), model_type,\
                  'seq{}_min{}'.format(seq_length1, min_contract_day), forecast_variable]
for model_dir in model_Dir_tree:
    if model_dir not in os.listdir(model_path):
        os.mkdir(model_path + model_dir)
    model_path = model_path + model_dir + '/'

loss_adj_columns_names = ['交易日期', '到期天數', '調整後loss']
loss_adj_columns_index = [Forecast_Data.columns.get_loc(col) for col in loss_adj_columns_names]
loss_adj_Data = Forecast_Data.iloc[range(0, len(Forecast_Data), K_num), loss_adj_columns_index]
loss_adj_Data = loss_adj_Data.reset_index().iloc[:,1:]
loss_adj_Data = loss_adj_Data.rename(columns={'調整後loss': model_name})
MSE_adj_data = pd.DataFrame(columns=loss_adj_Data.columns, data=[['MSE', 'MSE', np.mean(sse_everyday_adj)]])
loss_adj_Data = pd.concat([loss_adj_Data, MSE_adj_data], axis=0)
loss_adj_Data = loss_adj_Data.reset_index().iloc[:,1:]



if 'Modelloss.csv' in os.listdir(model_path):
    Modelloss = pd.read_csv(model_path + 'Modelloss.csv', index_col=False, encoding='Big5')
    Modelloss_adj = pd.read_csv(model_path + 'Modelloss_adj.csv', index_col=False, encoding='Big5')
    if model_name in Modelloss.columns:
        if Modelloss[model_name][len(Modelloss)-1] > np.mean(sse_everyday):
            Modelloss[model_name] = loss_Data[model_name]
            Modelloss_adj[model_name] = loss_adj_Data[model_name]
            Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
            model.save(model_path + model_file)
            plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=True)
            print('原本的{}已經被替換'.format(model_name))
        else:
            print('此模型未被儲存')
    elif len(Modelloss.columns) >= max_model+2 :
        all_MSE = np.array(Modelloss.iloc[-1, 2:])
        max_MSE = np.max(all_MSE)
        if max_MSE > np.mean(sse_everyday):
            max_MSE_idx = (np.arange(len(all_MSE))[np.equal(all_MSE, max_MSE)])[0] +2
            Modelloss.iloc[:,max_MSE_idx] = loss_Data[model_name]
            Modelloss_adj.iloc[:,max_MSE_idx] = loss_adj_Data[model_name]
            model_name = Modelloss.columns[max_MSE_idx]
            Forecast_name = model_name + '.csv'
            model_file = model_name + '.h5'
            model_png = model_name + '.png'
            Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
            model.save(model_path + model_file)
            plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=True)
            print('原本的{}已經被替換'.format(model_name))
        else:
            print('此模型未被儲存')
    else:
        Modelloss = pd.concat([Modelloss, loss_Data[model_name]], axis=1)
        Modelloss = Modelloss.reset_index().iloc[:,1:] 
        Modelloss_adj = pd.concat([Modelloss_adj, loss_Data[model_name]], axis=1)
        Modelloss_adj = Modelloss_adj.reset_index().iloc[:,1:] 
        Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
        model.save(model_path + model_file)
        plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=True)
        print('此模型已經被儲存為{}'.format(model_name))
else:
    Modelloss = loss_Data
    Modelloss_adj = loss_adj_Data
    Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
    model.save(model_path + model_file)
    plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=True)

    
Modelloss.to_csv(model_path + 'Modelloss.csv', index=False, encoding='Big5')
Modelloss_adj.to_csv(model_path + 'Modelloss_adj.csv', index=False, encoding='Big5')

Epoch 1/1000


: 